In [1]:
debug = False
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import glob
sys.path.append('../py/')
from s027_kfold_ods import ods_kfold
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

#========================================================================
# Args
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg']
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
#========================================================================

2019-02-20 11:40:47,512 utils 400 [INFO]    [logger_func] start 


In [2]:
base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target]].set_index(key)
df_hist = pd.read_csv('../input/historical_transactions.csv')
df_hist['purchase_amount_new'] = np.round(df_hist['purchase_amount'] / 0.00150265118 + 497.06, 2)
auth1 = df_hist[df_hist['authorized_flag']=='Y']
auth0 = df_hist[df_hist['authorized_flag']=='N']

In [158]:
tmp = auth1.copy()

use_cols = [key, 'month_lag', 'purchase_date', 'authorized_flag', 'installments', 'merchant_id', 'purchase_amount_new', 'subsector_id', 'merchant_category_id']
tmp = tmp[use_cols]

# First purchase_date amount
date_min = tmp.groupby([key, 'merchant_id'])['purchase_date'].min().reset_index()
date_min = tmp.merge(date_min, how='inner', on=[key, 'merchant_id', 'purchase_date'])
date_min = date_min[[key, 'merchant_id', 'purchase_amount_new']].rename(columns={'purchase_amount_new':'purchase_amount_new_first'})

# First purchase_month amount
month_min = tmp.groupby([key, 'merchant_id'])['month_lag'].min().reset_index()
month_min = tmp.merge(month_min, how='inner', on=[key, 'merchant_id', 'month_lag'])
month_min = month_min[[key, 'merchant_id', 'purchase_amount_new']].groupby([key, 'merchant_id'])['purchase_amount_new'].sum().reset_index().rename(columns={'purchase_amount_new':'purchase_amount_new_first_month'})

date_min.set_index([key, 'merchant_id'], inplace=True)
month_min.set_index([key, 'merchant_id'], inplace=True)
df_first_ratio = date_min.join(month_min)
print(df_first_ratio.shape)
display(df_first_ratio.head())

amount per first amount or first month amount
tmp.set_index([key, 'merchant_id'], inplace=True)
df_first_ratio = tmp.reset_index()[[key, 'merchant_id', 'month_lag', 'purchase_date', 'purchase_amount_new']].merge(df_first_ratio.reset_index(), on=[key, 'merchant_id'], how='inner')
df_first_ratio = utils.row_number(df=df_first_ratio, level=[key, 'merchant_id'])

df_first_ratio['purchase_amount_new_per_first_amount'] =  df_first_ratio['purchase_amount_new'] / df_first_ratio['purchase_amount_new_first']
df_first_ratio['purchase_amount_new_per_first_month_amount'] =  df_first_ratio['purchase_amount_new'] / df_first_ratio['purchase_amount_new_first_month']
df_first_ratio['purchase_amount_new_replace'] = df_first_ratio['purchase_amount_new'].replace(22.9, 27.9)

#========================================================================
# Ratio first purchase_amount ratio
#========================================================================
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target]].set_index(key)

df_rank2 = df_first_ratio[df_first_ratio['row_no']==2]
aggs = {}
aggs['purchase_amount_new_per_first_amount'] = ['mean', 'max', 'min']
aggs['purchase_amount_new_per_first_month_amount'] = ['mean', 'max', 'min']
new_cols = get_new_columns('second_purchase', aggs)
df_agg = df_rank2.groupby(key).agg(aggs)
df_agg.columns = new_cols
df_agg['second_purchase_purchase_amount_new_per_first_amount_max-min'] = df_agg['second_purchase_purchase_amount_new_per_first_amount_max'] - df_agg['second_purchase_purchase_amount_new_per_first_amount_min']
df_agg['second_purchase_purchase_amount_new_per_first_month_amount_max-min'] = df_agg['second_purchase_purchase_amount_new_per_first_month_amount_max'] - df_agg['second_purchase_purchase_amount_new_per_first_month_amount_min']
display(df_agg.head())

prefix = '257_one'
base = base.join(df_agg).fillna(-1)
utils.elo_save_feature(df_feat=base, prefix=prefix)

In [171]:
#========================================================================
# Ratio first purchase_amount nunique
#========================================================================
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target]].set_index(key)

mer_amount_nunique = df_first_ratio.groupby([key, 'merchant_id'])['purchase_amount_new_replace'].nunique()
amount_nunique = df_first_ratio.groupby([key])['purchase_amount_new_replace'].nunique()
mer_amount_nunique.name = 'merchant_amount_new_nunique'
amount_nunique.name = 'amount_new_nunique'
df_mer_nunique = mer_amount_nunique.reset_index().merge(amount_nunique.reset_index(), how='inner', on=key)

aggs = {}
aggs['merchant_amount_new_nunique'] = ['mean', 'max', 'min']
aggs['amount_new_nunique'] = ['mean']
new_cols = get_new_columns('variety', aggs)
df_agg = df_mer_nunique.groupby(key).agg(aggs)
df_agg.columns = new_cols
df_agg['variety_merchant_amount_new_nunique_max_per_total'] = df_agg['variety_merchant_amount_new_nunique_max'] / df_agg['variety_amount_new_nunique_mean']
display(df_agg.head())

prefix = '257_one'
base = base.join(df_agg).fillna(-1)
utils.elo_save_feature(df_feat=base, prefix=prefix)

In [18]:
#========================================================================
# そのmerchantを何か月連続で利用しているか
#========================================================================

# df = auth1.copy()
# df['yyyymm'] = df['purchase_date'].map(lambda x: str(x)[:7])
# df_month = df[[key, 'merchant_id', 'yyyymm']].drop_duplicates()
# del df
# gc.collect()
# df_month.sort_values(by=[key, 'merchant_id', 'yyyymm'], inplace=True)

df_month['yyyymm_shift1'] = df_month.groupby([key, 'merchant_id'])['yyyymm'].shift(1)
df_month['diff_month'] = ((pd.to_datetime(df_month['yyyymm']) - pd.to_datetime(df_month['yyyymm_shift1'])).dt.days)//28
# 連続してる時はフラグ1がたち、それ以外は0
df_month['continuours_flg'] = df_month['diff_month'].where(df_month['diff_month']==1, 0)

In [20]:
from tqdm import tqdm
# 0になったらしきり直しなので1, 1が続く限り+1していけば、idなど関係なく、1が続いた数をカウントできる。（天才的）
cont_list = df_month['continuours_flg'].values
continuous_list = []
for i in tqdm(cont_list):
    if i==0:
        cnt=1
    else:
        cnt+=1
    continuous_list.append(cnt)
print(len(cont_list))
print(len(continuous_list))

df_month['continuous'] = continuous_list
mer_dict = auth1[['merchant_id', 'subsector_id']].drop_duplicates().set_index('merchant_id').to_dict()['subsector_id']

100%|██████████| 18125053/18125053 [00:13<00:00, 1393641.85it/s]


18125053
18125053


In [34]:
#========================================================================
# all merchantにおける最大連続月回数
#========================================================================
cont_max = df_month.groupby([key, 'merchant_id'])['continuous'].max().reset_index()
cont_max['subsector_id'] =  cont_max['merchant_id'].map(mer_dict)
df_agg = cont_max.groupby(key)['continuous'].max()

base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target]].set_index(key)
base['merchant_continuous_month_max'] = df_agg
prefix = '257_one'
utils.elo_save_feature(df_feat=base, prefix=prefix)

In [43]:
#========================================================================
# 各subsectorにおける最大連続月回数
#========================================================================
# subsec_cont_max = cont_max.groupby([key, 'subsector_id'])['continuous'].max()
# subsec_cont_max = subsec_cont_max.unstack().fillna(-1)
# subsec_cont_max.columns = [f"continuous_month_max_subsector_id_{col}" for col in subsec_cont_max.columns]
base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target, 'hist_regist_term']].set_index(key)
base[subsec_cont_max.columns] = subsec_cont_max
utils.elo_save_feature(df_feat=base, prefix=prefix)

In [47]:
# from joblib import Parallel, delayed
# #========================================================================
# # Lift FAM * hist_regist_term
# term_name = 'hist_regist_term'
# term_list = sorted(list(base[term_name].value_counts().index))
# feat_cols = [col for col in base.columns if col.count('continuous')]

# def make_df_term(term):
#     tmp_list = []
#     for col in feat_cols:
#         tmp = base[base[term_name]==term].reset_index()
#         tmp = tmp[[key, col]]
#         tmp[col] = tmp[col] - tmp[col].min()
#         base_avg = tmp[col].mean()
#         tmp.set_index(key, inplace=True)
#         tmp[col] /= base_avg
#         tmp.rename(columns={col:f'lift_{term_name}_{col}'}, inplace=True)
#         tmp_list.append(tmp)
#     tmp_term = pd.concat(tmp_list, axis=1)
#     return tmp_term

# df_term_list = Parallel(n_jobs=-1)([delayed(make_df_term)(term) for term in term_list])
# df_term_lift = pd.concat(df_term_list, axis=0)
    
# tmp_base = utils.read_pkl_gzip('../input/base_no_out_clf.gz').set_index(key)['target'].to_frame()
# df_term_lift = tmp_base.join(df_term_lift)
# print(df_term_lift.shape)
# del tmp_base
# gc.collect()
# display(df_term_lift.head(10))
prefix = '258_lif'
utils.elo_save_feature(df_feat=df_term_lift, prefix=prefix)
#========================================================================